In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt
import os 
import re
from scipy.optimize import minimize

In [2]:
df_regions=pd.read_csv("/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/PEPR_Marseille/Selected subjects/For each subject/Evaluation_metrics_with_v4/Quantile for each subject/List_of_regions")

In [3]:
df_regions

,Unnamed: 0,index_subject,list of regions
0,0,5,"[('S.F.median-S.F.pol.tr.-S.F.sup.', 'R'), ('S..."
1,1,6,"[('S.Call.-S.s.P.-S.intraCing.', 'L'), ('S.s.P..."
2,2,14,"[('S.T.i.-S.T.s.-S.T.pol.', 'L'), ('S.T.i.-S.O..."
3,3,16,"[('F.Coll.-S.Rh.', 'L'), ('S.T.i.-S.T.s.-S.T.p..."
4,4,17,"[('S.F.median-S.F.pol.tr.-S.F.sup.', 'R'), ('S..."
...,...,...,...
114,114,685,"[('S.Call.', 'L'), ('S.Call.-S.s.P.-S.intraCin..."
115,115,690,"[('S.Po.C.', 'L'), ('S.C.-S.Po.C.', 'L'), ('F...."
116,116,694,"[('S.T.i.-S.T.s.-S.T.pol.', 'L'), ('Lobule_par..."
117,117,699,"[('S.F.median-S.F.pol.tr.-S.F.sup.', 'R'), ('S..."


In [4]:
dir_path = "/neurospin/tmp/fred/models/2025-11-10"

In [5]:
def find_subfolder_starting_with(c, c2):
    for name in os.listdir(c):
        full_path = os.path.join(c, name)
        if os.path.isdir(full_path) and name.startswith(c2):
            return full_path
    return None

In [6]:

selected_index = np.load("/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/PEPR_Marseille/All the subjects/histogram_non_zero/index_to_save.npy")

In [7]:
import ast

In [8]:
def find_auc(x):
    a,b,c=x

    auc_value=[]

    for i in df_regions["index_subject"]:

        y_true=[]
        global_score=[]

        L=df_regions.loc[df_regions["index_subject"] == i, "list of regions"].iloc[0]
        L = ast.literal_eval(L)

        ROI=[] #Regions Of Interest

        for all_region in L:
        
            side =all_region[1]
            region= all_region[0]

            region= region.replace(".","")

            if side=='L':
                region= region + "_left"
            if side=='R':
                region= region + "_right"

            dir_region = find_subfolder_starting_with(dir_path, region) #The path to the corresponding region

            ROI.append(dir_region)

        for subdir in os.listdir(dir_path):

            path_subdir=os.path.join(dir_path, subdir)

            if path_subdir in ROI:
                y_true.append(1)
            else:
                y_true.append(0)

            entropy_path = os.path.join(dir_path,subdir,"PEPR_Marseille", "Quantiles_Reconstruction_error.csv")
            chamfer_path = os.path.join(dir_path, subdir, "PEPR_Marseille", "Quantiles_Chamfer_distance.csv")
            SVM_path = os.path.join(dir_path,subdir, "PEPR_Marseille", "Quantiles_Residualisation_Score_SVM.csv")

            entropy_PEPR= pd.read_csv(entropy_path)
            chamfer_PEPR= pd.read_csv(chamfer_path)
            SVM_PEPR= pd.read_csv(SVM_path)

            entropy_PEPR= entropy_PEPR.loc[selected_index].reset_index(drop=True)
            chamfer_PEPR=chamfer_PEPR.loc[selected_index].reset_index(drop=True)
            SVM_PEPR=SVM_PEPR.loc[selected_index].reset_index(drop=True)


            global_score.append(a*entropy_PEPR.iloc[i,-1] + b*chamfer_PEPR.iloc[i,-1] +c*SVM_PEPR.iloc[i,-1])

        auc_value.append(roc_auc_score(y_true,global_score))
    auc_value=np.array(auc_value)

    return np.mean(auc_value)

In [9]:
constraints = (
    {"type": "eq", "fun": lambda x: x[0] + x[1] + x[2] - 1},
)

bounds = [(0, 1), (0, 1), (0, 1)]

In [10]:
x0 = np.array([1/3, 1/3, 1/3])  # point initial

res = minimize(
    lambda x: -find_auc(x),  # maximisation
    x0,
    bounds=bounds,
    constraints=constraints,
    method="SLSQP"
)

a_opt, b_opt, c_opt = res.x
f_max = find_auc(res.x)

In [11]:
print("Maximum de f :", f_max)
print("a =", a_opt)
print("b =", b_opt)
print("c =", c_opt)

Maximum de f : 0.5489837088308935
a = 0.3332479219480503
b = 0.3326829232084114
c = 0.3340691983762295
